# Boston Housing Linear Regression

In [ ]:
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from helpers.helper_functions import translate_to_file_string

Select the Imput File

In [ ]:
inputFile = translate_to_file_string("../data/Boston_Housing_Data.csv")

SparkSession creation

In [ ]:
spark = (SparkSession
       .builder
       .appName("BostonHousingRegression")
       .getOrCreate())

Create a DataFrame using an ifered schema 

In [ ]:
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ";") \
       .csv(inputFile)
print(df.printSchema())

Select the features

In [ ]:
featureCols = df.columns.copy()
featureCols.remove("MEDV")
featureCols.remove("CAT")   
assembler =  VectorAssembler(outputCol="features", inputCols=featureCols)

Data Preparation

In [ ]:
labeledPointDataSet = assembler.transform(df)

Prepare training and test data.

In [ ]:
splits = labeledPointDataSet.randomSplit([0.9, 0.1 ], 12345)
training = splits[0]
test = splits[1]

Build and train the Model

In [ ]:
# TODO Optimize the LR Model 
lr = LinearRegression(maxIter=1000, regParam=0.0, elasticNetParam=0.0, loss='squaredError', featuresCol="features", labelCol="MEDV")
lrModel = lr.fit(training)
# Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

Test the Model

In [ ]:
predictionsLR = lrModel.transform(test)
predictionsLR.show()

In [ ]:
evaluator = RegressionEvaluator(labelCol="MEDV",predictionCol="prediction", metricName="rmse")

In [ ]:
print("root mean square error = " , evaluator.evaluate(predictionsLR))
spark.stop()